In [ ]:
import pandas as pd
import numpy as np
import math
import re
import datetime as dt
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

In [ ]:
dogs = pd.read_csv("/Users/paky/Desktop/Uni/Foundations of computer science/Progetto/adoptions/dogs.csv")
dogtravel = pd.read_csv("/Users/paky/Desktop/Uni/Foundations of computer science/Progetto/adoptions/dogTravel.csv")
pop = pd.read_csv("/Users/paky/Desktop/Uni/Foundations of computer science/Progetto/adoptions/NST-EST2021-POP.csv" , header = None)

In [ ]:
# FIX CSV

# Questa funzione controlla se ci sono numeri in una stringa

def has_numbers(inputString): 
    return bool(re.search(r'\d', inputString))
id_problematici = []
k = 0

# Se ci sono lettere nella variabile posted (dovrebbe essere una data) l'id viene aggiunto in una lista

for i in dogs["posted"]:
    if has_numbers(dogs["posted"][k]) == True: 
        pass
    else:
        id_problematici.append([dogs["id"][k]])
    k+=1
    
# Shift di variabili per le righe con problemi così che in "posted" ci sia una data e non una città
    
for i in range(len(dogs["id"])):
  if dogs["id"][i] in id_problematici:
    dogs.iloc[i:i+1,25:34] = dogs.iloc[i:i+1,25:34].shift(periods = 1, axis = 1)
  else:
    pass

In [ ]:
## 1) Extract all dogs whose status is different from adoptable

dogs[dogs["status"] != "adoptable"]

In [ ]:
## 2) For each primary breed, determine the number of dogs

dogs.groupby("breed_primary").count()["id"]

In [ ]:
## 3) For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed.

# Per ogni primary_breed viene effettuato un rapporto tra i cani che hanno "Mixed Breed" come secondary_breed e coloro che non presentano tale valore

dogs[dogs['breed_secondary'] == 'Mixed Breed'].groupby("breed_primary").count()["id"]/dogs[dogs['breed_secondary'] != 'Mixed Breed'].groupby("breed_primary").count()["id"]

# NaN Values are determined by the absence of secondary mixed breed.

In [ ]:
## 4) For each (primary) breed, determine the earliest and the latest posted timestamp.

# Trasformare posted in datetime

dogs["posted"] = pd.to_datetime(dogs["posted"])

# raggruppare per razza

group = dogs.groupby(['breed_primary'])

# Trovare i timestamp postati per primi e per ultimi

result = group.agg({ 'posted' : [np.min, np.max]})
result.columns = result.columns.get_level_values(0)
result.columns = ["earliest","latest"]
result

In [ ]:
# 5) For each state, compute the sex imbalance, that is the difference between male and female dogs. 
#    In which state this imbalance is largest?

# Per ogni stato veiene calcolata la differenza tra il numero di cani con variabile "sex" uguale a "Male" e il numero di cani con variabile "sex" uguale a "Female"

imb = pd.DataFrame(dogs[dogs['sex'] == 'Male'].groupby("contact_state").count()["id"] - dogs[dogs['sex'] == 'Female'].groupby("contact_state").count()["id"])
imb.rename(columns = {'id':'imbalance'}, inplace = True)
imb

In [ ]:
print("The state with the maximum imbalance is:", str(imb.index[imb['imbalance']== imb["imbalance"].max()][0]))

In [ ]:
# 6) For each pair (age, size), determine the average duration of the stay and the average cost of stay.

# Viene effettuato un doppio group by sulle variabili "age" e "size" alle quali vengono affiancati i valori medi per "stay_duration" e "stay_cost" relativi ad ogni categoria delle due colonne raggruppate.

dogs.groupby(["age","size"]).mean()[["stay_duration","stay_cost"]]

In [ ]:
# 7) Find the dogs involved in at least 3 travels. Also list the breed of those dogs. 

#id dei due database sono trasformati in stringhe
dogtravel["id"].apply(str)
dogs["id"].apply(str)

# gli id vengono impostati come indici  
dogtravel.set_index('id')
dogs.set_index("id")

# tabella dei cani che hanno effettuato almeno 3 viaggi
tab1 = dogtravel.groupby("id").count()[dogtravel.groupby("id").count()["index"]>=3]["index"]

# join tra la tabella appena creata e il database dei cani per trovare la razza 
result9 = pd.merge(tab1, dogs[["id","breed_primary"]], left_on = "id", right_on = "id")
result9 = result9.rename(columns = {"index":"Travels"})
result9

In [ ]:
# 8) Fix the travels table so that the correct state is computed from the manual and the found fields.
#    If manual is not missing, then it overrides what is stored in found.

# Prima si fa un controllo su "manual"
# Se manual non è missing si va a sostituire il valore in "found" con quello di "manual"

dogtravel8 = dogtravel
for i in range(len(dogtravel["manual"])):
    if pd.isnull(dogtravel["manual"][i]) == True:
        pass
    else:
        dogtravel8["found"][i] = dogtravel["manual"][i]
dogtravel8

In [ ]:
# 9) For each state, compute the ratio between the number of travels and the population.

# Prima si importa un dizionario contenente tutte le sigle degli stati

states = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

states = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}

# Si crea una nuova tabella con i codici degli stati, i nomi e la popolazione

pop.columns = ["col1","col2"]
pop["codes"] = states.keys()
pop["states"] = states.values()
pop["population"] = pop.iloc[:,1]
pop.drop(columns = ["col1","col2"], inplace = True)
for i in range(len(dogtravel["contact_state"])):
    if dogtravel["contact_state"][i] == "17325":
        dogtravel["contact_state"][i] = "PA"
    else:
        pass
pop

In [ ]:
# Nuova tabella in cui per ogni stato si contano il numero di viaggi

q9t1 = dogtravel.groupby(["contact_state"]).count()["index"]
q9t1 = pd.DataFrame(q9t1)
q9t1.reset_index(level=0, inplace=True)
q9t1.columns = ["codes","travels"]

# Join sugli stati tra la tabella creata in precedenza e quest'ultima, aggiungendo una colonna con il rapporto tra viaggi e popolazione

q9r1 = pd.merge(pop,q9t1,left_on="codes",right_on="codes",how="left")
for i in range(len(q9r1["population"])):
    q9r1["population"][i] = int(q9r1["population"][i].replace(".",""))
q9r1["ratio"] = q9r1["travels"] / q9r1["population"]
q9r1["ratio"] = q9r1["ratio"].replace(np.nan, 0).round(6)
q9r1["travels"] = q9r1["travels"].replace(np.nan, 0).astype(int)
q9r1

In [ ]:
# 10) For each dog, compute the number of days from the posted day to the day of last access.

# Prima si trasformano le variabili posted e accessed in datetime

dogs["posted"] = pd.to_datetime(dogs["posted"])
dogs["posted"] = dogs["posted"].dt.date
dogs["accessed"] = pd.to_datetime(dogs["accessed"])
dogs["accessed"] = dogs["accessed"].dt.date

# Calcolo dei giorni passati da "accessed" a "posted"

passed_days = dogs["accessed"] - dogs["posted"]
dogs1 = dogs
dogs1["passed_days"] = passed_days
result10 = dogs1[["id","passed_days"]]
HTML(result10.to_html(index=False))

In [ ]:
# 11) Partition the dogs according to the number of weeks from the posted day to the day of last access.

# Prima si trasformano i giorni della tabella precedente in settimane 

table11 = result10
table11['weeks'] = table11['passed_days']/np.timedelta64(1,'W')

# Si arrotonda per difetto, quindi meno di 7 giorni vengono considerati come 0 settimane  

table11['weeks'] = (table11['weeks']).apply(np.floor)
table11['weeks'] = table11['weeks'].astype(int)

# Si raggruppano i cani in base al numero di settimane 

result11 = table11[["id","weeks"]]
result11 = result11.groupby(["weeks","id"]).count()
result11

In [ ]:
# 12) Find for duplicates in the dogs dataset.

similardogs = dogs.groupby(["sex","breed_primary","breed_secondary"]).count() #group by similar dogs according to sex and primary and secondary breed
similardogs = pd.DataFrame(similardogs[similardogs['id']>1]['id']) #select the dogs with same breeds and sex that appear more than once
similardogs.reset_index(inplace = True)  #reset indices
similardogs

In [ ]:
df_sec_breed = []

#tramite un ciclo for racchiudi in una lista tutti i dataframe contenenti cani con stesso sesso, primary e secondary breed.
for x in range(similardogs.shape[0]): 
  df = dogs.loc[(dogs['sex']==similardogs.loc[x]['sex']) & (dogs['breed_primary']==similardogs.loc[x]['breed_primary'])&(dogs['breed_secondary']==similardogs.loc[x]['breed_secondary'])]
  df_sec_breed.append(df[["id","description"]])
    
from difflib import SequenceMatcher


# funzione per la similarità: ratio restituisce un punteggio di similarità tra 0 e 1 tra due stringhe.
# ratio è calcolato come: 2M/T; dove T è il numero totale di elementi ed M è il numero totale di match.
def similar(a, b):
  if pd.isnull(a) or pd.isnull(b):
    return 0
  else:
    return SequenceMatcher(None, a, b).ratio()

from itertools import combinations


#per ogni dataframe contenuto in df_sec_breed viene calcolata la somiglianza per ogni coppia possibile in quel dataframe.
for x in df_sec_breed:
  for index in list(combinations(x.index,2)):
    paired = pd.DataFrame(x.loc[index,:])
    if similar(paired.iloc[0]['description'], paired.iloc[1]['description']) >= 0.90:
      print(f"dog {paired.iloc[0]['id']} and dog {paired.iloc[1]['id']} are duplicates. Similarity is {similar(paired.iloc[0]['description'], paired.iloc[1]['description'])}")
    else:
      pass
print(x)